In [ ]:
'''데이터 생성 스크립트'''
# bboxes = data['bboxes'] # nested array: [[x, y, w, h],...]
# scores = data['scores'] # [0.6, 0.89,...]
# labels = data['labels'] # [0, 4,...] # yolo 라벨 0-23
# timestamp = data['timestamp'] # unix
# width = data['width'] # 이미지 width
# height = data['height'] # 이미지 height
# region_and_name = data['region_and_name'] # 카메라 정보. id?
# img_directory = data['img_directory'] # 이미지 저장 경로
# dict_values([[[580.052604675293, 217.65965223312378, 60.063819885253906, 58.98589611053467], [330.85792541503906, 175.94241857528687, 33.85063171386719, 39.26307678222656]], [0.6402683258056641, 0.5894748568534851], [0, 4], 1693530106898, 640, 360, '지역', 'C:/cctv_images/지역_1693530106898.jpg'])

In [96]:
import pandas as pd

columns = ['bboxes', 'scores', 'labels', 'timestamp', 'width', 'height', 'cctvid', 'region_and_name', 'centername', 'img_directory']

df = pd.DataFrame(columns=columns)
df

,bboxes,scores,labels,timestamp,width,height,cctvid,region_and_name,centername,img_directory


In [42]:
cctv_df = pd.read_csv(r'static\CCTV.csv')
cctv_df.head()

,Unnamed: 0,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,URL
0,0,1,L280034,가덕영업소,거제교통정보센터,128.817344,35.025620,https://www.utic.go.kr/view/map/openDataCctvSt...
1,1,2,L280005,거가1교,거제교통정보센터,128.731926,35.017735,https://www.utic.go.kr/view/map/openDataCctvSt...
2,2,3,L280003,거가1교(저도방향),거제교통정보센터,128.721416,35.017084,https://www.utic.go.kr/view/map/openDataCctvSt...
3,3,4,L280054,거가2교,거제교통정보센터,128.758293,35.014669,https://www.utic.go.kr/view/map/openDataCctvSt...
4,4,5,L280001,거제면 귀목정,거제교통정보센터,128.581956,34.862783,https://www.utic.go.kr/view/map/openDataCctvSt...


In [43]:
cctv_df.shape

(7746, 8)

In [118]:
import random
import time
import os
class GenerateRandomData:
    def __init__(self, x1=None, y1=None, score=None, label=None, width=None, height=None, cctvid=None, img_directory_pre=None):
        '입력 안 하면 랜덤으로 x1, y1, label 설정됨'
        self._width = width
        self._height = height
        self._generate_width_height()
        self.x1 = x1
        self.y1 = y1
        if not all([self.x1, self.y1]):
            self.x1, self.y1 = self.generate_starting_point()
        self.bbox = self.generate_bbox(self.x1, self.y1)
        self.score = score or self.generate_score()
        self.label = label or self.generate_label()
        self._cctvid = cctvid or self._generate_region_and_name()[0]
        self._img_directory_pre = img_directory_pre or self._generate_img_directory_pre()
        self.img_directory = os.path.join(self._img_directory_pre, self._cctvid + '_' + str(int(time.time()*1000)) + '.jpg')

    def generate_starting_point(self):
        x1 = random.uniform(0, self._width)
        y1 = random.uniform(0, self._height)
        return x1, y1

    def generate_bbox(self, x1, y1):
        '''
        x,y left-top coordinates로 bbox 만들기\n
        x1, x2, y1, y2로 구성
        '''
        bwidth = random.uniform(10,100)
        bheight = random.uniform(10,100)
        x2 = x1 + bwidth
        y2 = y1 + bheight
        # 이미지 사이즈 안 넘어가게
        if x2 > self._width: x2 = self._width
        if y2 > self._height: y2 = self._height
        return [self.x1, x2, self.y1, y2]
    
    def generate_score(self):
        'score 생성'
        return random.uniform(0.5, 1)
    
    def generate_label(self):
        'label 생성'
        return random.randrange(0, 23)
    
    def _generate_width_height(self, new=False):
        'width, height 생성'
        self._width = random.choice([640, 1280, 1600, 1980]) if new else (self._width if self._width else random.choice([640, 1280, 1600, 1900]))
        self._height = int(self._width * (9/16)) if new else (self._height if self._height else int(self._width * (9/16)))
        return self._width, self._height
    
    def generate_timestamp(self):
        'timestamp 생성'
        return int(time.time()*1000)

    def _generate_region_and_name(self, new=False):
        'region 정보 생성'
        idx = random.randrange(0, 7746)
        cctvid, cctvname, centername = cctv_df.iloc[idx, 2:5]
        return [cctvid, cctvname, centername]
    
    def _generate_img_directory_pre(self, new=False):
        '파일 부모 저장경로 생성'
        return random.choice([r'C:/cctv_images', r'/home/cctv_images', r'~/Downloads/cctv_images'])
    
    def generate_img_directory(self, timestamp):
        '파일 저장경로 생성'
        return os.path.join(self._img_directory_pre, self._cctvid + '_' + str(timestamp) + '.jpg')
    
    def generate_bulk_data(self, count=100):
        'bulk data 생성'
        dict_ = {column: [] for column in columns}
        # width, height 생성
        width, height = self._generate_width_height(new=True)
        # cctv 정보 생성
        cctvid, cctvname, centername = self._generate_region_and_name(new=True)
        # img_directory_pre 생성
        img_directory_pre = self._generate_img_directory_pre(new=True)
        for _ in range(count):
            # columns = ['bboxes', 'scores', 'labels', 'timestamp', 'width', 'height', 'cctvid', 'region_and_name', 'centername', 'img_directory']
            # x1,y1 생성
            x1, y1 = self.generate_starting_point()
            # bbox 생성
            bbox = self.generate_bbox(x1, y1)
            # score 생성
            score = self.generate_score()
            # label 생성
            label = self.generate_label()
            # timestamp 생성
            timestamp = self.generate_timestamp()
            # img_directory 생성
            img_directory = os.path.join(img_directory_pre, cctvid + '_' + str(timestamp) + '.jpg')
            dict_['bboxes'].append([bbox])
            dict_['scores'].append([score])
            dict_['labels'].append([label])
            dict_['timestamp'].append(timestamp)
            dict_['width'].append(width)
            dict_['height'].append(height)
            dict_['cctvid'].append(cctvid)
            dict_['region_and_name'].append(cctvname)
            dict_['centername'].append(centername)
            dict_['img_directory'].append(img_directory)
        return pd.DataFrame.from_dict(dict_)

generator = GenerateRandomData()
generator.generate_label()
generator._generate_region_and_name()
df_ = generator.generate_bulk_data()

In [122]:
df = pd.DataFrame(columns=columns)

for _ in range(1000):
    df = pd.concat([df, generator.generate_bulk_data(100)])
df

,bboxes,scores,labels,timestamp,width,height,cctvid,region_and_name,centername,img_directory
0,"[[1248.2773209596248, 1725.8516934325214, 629....",[0.8833946827175737],[2],1693537473948,1980,1113,L310097,중앙초교사거리,광주광역시,C:/cctv_images\L310097_1693537473948.jpg
1,"[[1248.2773209596248, 666.3318756399414, 629.1...",[0.9061172555964929],[2],1693537473948,1980,1113,L310097,중앙초교사거리,광주광역시,C:/cctv_images\L310097_1693537473948.jpg
2,"[[1248.2773209596248, 420.4772879845491, 629.1...",[0.6330994195120204],[8],1693537473948,1980,1113,L310097,중앙초교사거리,광주광역시,C:/cctv_images\L310097_1693537473948.jpg
3,"[[1248.2773209596248, 1895.9268208942333, 629....",[0.5967716546861244],[7],1693537473948,1980,1113,L310097,중앙초교사거리,광주광역시,C:/cctv_images\L310097_1693537473948.jpg
4,"[[1248.2773209596248, 970.8410670659739, 629.1...",[0.725714953342883],[5],1693537473948,1980,1113,L310097,중앙초교사거리,광주광역시,C:/cctv_images\L310097_1693537473948.jpg
...,...,...,...,...,...,...,...,...,...,...
95,"[[1248.2773209596248, 328.19511409791005, 629....",[0.5730578905857237],[11],1693537481853,1280,720,E900901,[국도5호선]효령삼거리,국가교통정보센터(국도),/home/cctv_images\E900901_1693537481853.jpg
96,"[[1248.2773209596248, 503.76536084820833, 629....",[0.9198962766066889],[14],1693537481853,1280,720,E900901,[국도5호선]효령삼거리,국가교통정보센터(국도),/home/cctv_images\E900901_1693537481853.jpg
97,"[[1248.2773209596248, 1142.6782126259059, 629....",[0.7677384221764565],[18],1693537481853,1280,720,E900901,[국도5호선]효령삼거리,국가교통정보센터(국도),/home/cctv_images\E900901_1693537481853.jpg
98,"[[1248.2773209596248, 1181.0599742594084, 629....",[0.9999706861796004],[18],1693537481853,1280,720,E900901,[국도5호선]효령삼거리,국가교통정보센터(국도),/home/cctv_images\E900901_1693537481853.jpg


In [133]:
type(df.iloc[0,0])

list

In [123]:
df.to_csv('generated_data2.csv', index=False)